In [1]:
import pandas as pd
import numpy as np

In [4]:
rating_data=pd.io.parsers.read_csv("ratings.dat",names=["userId","movie_id","rating","timestamp"],engine="python",delimiter="::")
movie_data=pd.io.parsers.read_csv("movies.dat",names=["movie_id","title","genre"],engine="python",delimiter="::")
movie_data

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


CREATING RATING MATRIX OF SHAPE mxu

In [5]:
#converting our data into matrix
rating_matrix=np.ndarray(shape=(np.max(rating_data.movie_id.values),np.max(rating_data.userId.values)))
rating_matrix[rating_data.movie_id.values-1,rating_data.userId.values-1]=rating_data.rating.values

In [6]:
rating_matrix

array([[5., 0., 0., ..., 0., 0., 3.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

SUBSTRACT MEAN OFF-NORMALISATION SCALING OUR DATA

In [11]:
normalized_matrix=rating_matrix-np.array([(np.mean(rating_matrix,1))]).T
normalized_matrix

array([[ 3.57400662, -1.42599338, -1.42599338, ..., -1.42599338,
        -1.42599338,  1.57400662],
       [-0.37152318, -0.37152318, -0.37152318, ..., -0.37152318,
        -0.37152318, -0.37152318],
       [-0.23874172, -0.23874172, -0.23874172, ..., -0.23874172,
        -0.23874172, -0.23874172],
       ...,
       [-0.03278146, -0.03278146, -0.03278146, ..., -0.03278146,
        -0.03278146, -0.03278146],
       [-0.02582781, -0.02582781, -0.02582781, ..., -0.02582781,
        -0.02582781, -0.02582781],
       [-0.24288079, -0.24288079, -0.24288079, ..., -0.24288079,
        -0.24288079, -0.24288079]])

computing svd

In [6]:
A=normalized_matrix.T/np.sqrt(rating_matrix.shape[0]-1)
U,D,V=np.linalg.svd(A)

In [7]:
def cosine_similarity(rating_data,movie_id,top=10):
    index=movie_id-1 #bcz i need index to start from 1
    movie_row=rating_data[index,:] #row is index and column i need full
    magnitude=np.sqrt(np.einsum("ij,ij->i",rating_data,rating_data))
    similar=np.dot(movie_row,rating_data.T)/(magnitude[index]*magnitude)
    sort_index=np.argsort(-similar)
    return sort_index[:top]

In [8]:
def sim_movies(movie_data,movie_id,top_indices):
    print("recomendations for {}".format(movie_data[movie_data.movie_id==movie_id].title.values[0]))
    for id in top_indices+1:
        print(movie_data[movie_data.movie_id== id].title.values[0])

In [9]:
k=50;movie_id=1;top=10
sliced=V.T[:,:k]
index=cosine_similarity(sliced,movie_id,top)

<ipython-input-7-51edd3244c91>:5: RuntimeWarning: invalid value encountered in true_divide
  similar=np.dot(movie_row,rating_data.T)/(magnitude[index]*magnitude)


In [10]:
sim_movies(movie_data,movie_id,index)

recomendations for Toy Story (1995)
Toy Story (1995)
Toy Story 2 (1999)
Babe (1995)
Bug's Life, A (1998)
Pleasantville (1998)
Babe: Pig in the City (1998)
Aladdin (1992)
Stuart Little (1999)
Secret Garden, The (1993)
Tarzan (1999)
